In [0]:
# Parameters: Year, Month
dbutils.widgets.text("Year", "")
dbutils.widgets.text("Month", "")

year = int(dbutils.widgets.get("Year"))
month = int(dbutils.widgets.get("Month"))

if year == 0 or month == 0:
    dbutils.notebook.exit("Idle")

In [0]:
# Date Range
import calendar
from datetime import date

start_date = date(year, month, 1)
end_date = date(year, month, calendar.monthrange(year, month)[1])
date_range = f"{start_date.strftime('%Y%m%d')}-{end_date.strftime('%Y%m%d')}"


In [0]:
# Source files
blob_account_name = "blob_account_name"
blob_container_name = "blob_container_name"
billingAccounts = "billingAccounts"
blob_relative_path = f"providers/Microsoft.Billing/billingAccounts/{billingAccounts}/{date_range}/amortizedcost/"
blob_sas_token = dbutils.secrets.get("sas_token_name", "accesskey")

In [0]:
wasbs_path = f"abfss://{blob_container_name}@{blob_account_name}.dfs.core.windows.net/{blob_relative_path}"
spark.conf.set(f"fs.azure.account.key.{blob_account_name}.dfs.core.windows.net", blob_sas_token)

In [0]:
# Table & View
raw_azure_view = "raw_azure_view"
raw_azure = "raw_azure"
azure_mca_view = "azure_mca_view"
azure_data = "azure_data"

In [0]:
# Get Raw Data from Source files
df = spark.sql(f"""
  SELECT * FROM parquet.`{wasbs_path}`
""").select("AdditionalInfo", "BenefitId", "BenefitName", "BillingAccountId", "BillingAccountName", "BillingCurrency", "BillingPeriodEndDate", "BillingPeriodStartDate", "BillingProfileId", "BillingProfileName", "ChargeType", "ConsumedService", "CostAllocationRuleName", "CostCenter", "CostInBillingCurrency", "CostInPricingCurrency", "CostInUsd", "Date", "EffectivePrice", "ExchangeRateDate", "ExchangeRatePricingToBilling", "Frequency", "InvoiceId", "InvoiceSectionId", "InvoiceSectionName", "IsAzureCreditEligible", "Location", "MeterCategory", "MeterId", "MeterName", "MeterRegion", "MeterSubCategory", "PaygCostInBillingCurrency", "PaygCostInUsd", "PayGPrice", "PreviousInvoiceId", "PricingCurrency", "PricingModel", "ProductId", "ProductName", "ProductOrderId", "ProductOrderName", "Provider", "PublisherId", "PublisherName", "PublisherType", "Quantity", "ResellerMpnId", "ResellerName", "ReservationId", "ReservationName", "ResourceGroupName", "ResourceId", "ResourceLocation", "ServiceFamily", "ServiceInfo1", "ServiceInfo2", "ServicePeriodEndDate", "ServicePeriodStartDate", "SubscriptionId", "SubscriptionName", "Tags", "Term", "UnitOfMeasure", "UnitPrice")

df.createOrReplaceTempView(raw_azure_view)

In [0]:
spark.sql(f"""
DELETE FROM {raw_azure} WHERE DATE_TRUNC('month', Date)::DATE = '{start_date}'
""")

In [0]:
df.write.format("delta").mode("append").option("delta.columnMapping.mode", "name").saveAsTable(raw_azure)

In [0]:
# Cache Raw Data from Source files
spark.sql(f"""
  SELECT SubscriptionId, SubscriptionName, ResourceGroupName AS ResourceGroup, ResourceLocation, Date::DATE AS Date, CostInBillingCurrency::DOUBLE AS CostInBillingCurrency, MeterCategory, PublisherName,
  -- Tags
  CASE WHEN Tags:sc_system = '' THEN NULL ELSE Tags:sc_system END AS sc_system,
  CASE WHEN Tags:sc_component = '' THEN NULL ELSE Tags:sc_component END AS sc_component,
  CASE WHEN Tags:sc_env = '' THEN NULL ELSE Tags:sc_env END AS sc_env,
  CASE WHEN Tags:sc_region = '' THEN NULL ELSE Tags:sc_region END AS sc_region,
  CASE WHEN Tags:sc_type = '' THEN NULL ELSE Tags:sc_type END AS sc_type,
  CASE WHEN Tags:sc_costowner = '' THEN NULL ELSE Tags:sc_costowner END AS sc_costowner,
  Tags:sc_createdby, Tags:sc_provider, 
  --
  PricingModel, InvoiceSectionName, YEAR(Date) AS Year, MONTH(Date) AS Month
  FROM {raw_azure_view}
""").createOrReplaceTempView(azure_mca_view)

In [0]:
spark.sql(f"DELETE FROM {azure_data} WHERE Year = {year} AND Month = {month}")

In [0]:
# Append Data
spark.sql(f"""
    INSERT INTO {azure_data}
    SELECT 
    --Tags fields
    u.SubscriptionId, u.SubscriptionName, u.SubscriptionName AS SubAccount, u.ResourceGroup, u.ResourceLocation,
    --System fields
    u.Date, u.CostInBillingCurrency AS Cost, u.PricingModel, u.InvoiceSectionName, 
    scta.CostType AS CostType, apsm.`System Name` AS System, apsm.`Product Name` AS Product, 'Azure' AS Vendor, u.MeterCategory AS VendorService,
    u.PublisherName AS Publisher,
    --Tags Json Fields
    u.sc_system AS sc_system,
    u.sc_component AS sc_component,
    u.sc_env AS sc_env,
    u.sc_region AS sc_region,
    u.sc_type AS sc_type,
    u.sc_costowner AS sc_costowner,
    u.sc_createdby AS sc_createdby,
    u.sc_provider AS sc_provider,
    u.Year, u.Month
    FROM {azure_mca_view} u
    LEFT JOIN subscription_cost_type_allocation scta 
        ON scta.SubscriptionName = u.SubscriptionName AND scta.InvoiceSectionName = u.InvoiceSectionName
    LEFT JOIN azure_product_system_mapping apsm 
        ON apsm.`System Name` = scta.SystemName
""")

In [0]:
dbutils.notebook.exit("OK")